In [181]:
import boto3
import requests
import pandas as pd
import json
import logging
import time
import re

from boto3.dynamodb.conditions import Key, Attr

from bs4 import BeautifulSoup
import requests

### 목표

    1. 고객이 조사하고 싶은 카테고리를 등록하면, 해당 카테고리의 Popular-Brand 리스트들을 추가
    2. 그 리스트들을 DynamoDB에 등록

In [201]:
# 등록된 카테고리에 대한 Popular-brand를 뽑아주는 class
# argument : 
#     - category_list : (cat_id, cat_title) 로 구성된 리스트 
# output : 
#     - brand_list : (brand_id, brand_title) 로 구성된 리스트
class BrandCrawler():
    base_url = 'https://search.shopping.naver.com/search/category.nhn'
    headers = {
        "accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-encoding":"gzip, deflate, br",
        "accept-language":"ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "user-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"}
    params = {
        'cat_id':0,
        "pagingIndex":1,
        'pagingSize':20,
        "viewType":'list',
        'sort':'date',
        'frm':'NVSHBRD'
    }
    method = "get"
    logger = logging.getLogger("brand_crawl")

    def __init__(self, category_list,**kwargs):
        self.research_list = category_list
        self.delay = kwargs.get('delay',0.5)

    def run(self):
        global EXIT
        self.logger.info('start to run')
        result_list = []
        
        for cat_id, cat_title in self.research_list:
            row = {}
            self.params['cat_id'] = cat_id
            res_text, res_url = self._requests_text()
            if res_text is None:
                continue
                
            row['cat_id'] = cat_id
            row['cat_title'] = cat_title      
            row['brand_list'] = self._parse_and_save(res_text, res_url) 
            result_list.append(row)
        self.logger.info('end to run')
        return result_list

    def _requests_text(self):
        try:
            res = requests.request(self.method,
                                   self.base_url,
                                   params=self.params,
                                   headers=self.headers,
                                   timeout=5)
            res_text = res.text
            res_url = res.url
        except requests.ConnectionError as e:
            self.logger.error(e)
            res_text = None
            res_url = None
        except requests.Timeout as e:
            self.logger.error(e)
            res_text = None
            res_url = None
        except requests.RequestException as e:
            self.logger.error(e)
            res_text = None
            res_url = None
        finally:
            return res_text, res_url

    def _parse_and_save(self, res_text, res_url):
        bsObj = BeautifulSoup(res_text,"html.parser")
        brand_div = bsObj.find("div",{"class":re.compile("brand_filter")})
        brand_hot = brand_div.find("ul",{'class':re.compile("finder_tit")})
        
        row = {}  
        row_list = []
        for brand_sec in brand_hot.find_all('li'):
            try:
                brand_title = (brand_sec.a.attrs['title'])
                brand_id = (brand_sec.a.attrs['data-filter-value'])
                row_list.append((brand_id, brand_title))
            except AttributeError as e:
                self.logger.error("brand_name or brand_id is missing... url : {}".format(res_url))
                continue
        return row_list

### 1. DynamoDB 생성하고, 데이터를 추가하기

In [33]:
dynamodb = boto3.resource("dynamodb", region_name="ap-northeast-2")

#### 1. 테이블 생성하기

**'users'라는 이름의 테이블을 생성**

- key schema
    - user - 파티션 키. 문자열에 대한 AttributeType : S
    - category_id - 정렬키. 숫자에 대한 AttributeType : N
- value schema
    - brands - (brand_name , brand_id)의 json 집합

**Capacity 설정**

- ReadCapcityUnits:
    - 필요한 읽기 용량 유닛 수* = 초당 읽기 항목 수 * 항목 크기(4KB 블록)
- WriteCapacityUnits:
    - 필요한 쓰기 용량 유닛 수 = 초당 쓰기 항목 수 * 항목 크기(1KB 블록)
    
    

In [72]:
table = dynamodb.create_table(
    TableName="users",
    KeySchema=[
        {
            "AttributeName" : "user",
            "KeyType" : "HASH"
        },
        {
            "AttributeName" : "category_id",
            "KeyType" : "RANGE"
        },
    ],
    AttributeDefinitions=[
        {
            "AttributeName":"user",
            "AttributeType":"S"
        },
        {
            "AttributeName":"category_id",
            "AttributeType":"N"
        },
    ],
    ProvisionedThroughput={
        "ReadCapacityUnits" : 5,
        "WriteCapacityUnits" : 5,
    }

)

#### 2. 데이터 추가하기

In [249]:
# 카테고리 이름을 crawl_category_list에 추가하면, 
# 해당 카테고리의 하위 말단 카테고리의 ID와 NAME을 
# target_category_list에 담음
crawl_category_list = ['서재/사무용가구'] # 가져올 카테고리의 이름

In [280]:
# 가져올 데이터를 수집하기
cat_df = pd.read_csv("./data/category.csv",sep=";",dtype=str)

target_category_list = set()
for _, row in cat_df.iterrows():
    # crawl_category_list에 있는 카테고리가 row에 있는 경우를 check
    if row.isin(crawl_category_list).sum() > 0:
        # 말단 category의 id와 이름을 담음
        cat_id = row[~row.isnull()][-2]
        cat_title = row[~row.isnull()][-1]
        target_category_list.add((cat_id,cat_title))
        
target_category_list = list(target_category_list)

# target_category_list에 해당하는 popular 브랜드 정보를 담음.
bcp = Brand_crawl_parser(target_category_list)
popular_brand_list = bcp.run()

In [282]:
# 데이터를 DynamoDB에 Batch Insert하기

table = dynamodb.Table("users")
with table.batch_writer() as batch:
    for item in popular_brand_list:
        batch.put_item(
            Item={
                "user" : "데스커",
                "category_id" : int(item['cat_id']),
                "category_title" : item['cat_title'],
                "brands" : json.dumps(item['brand_list'])
            }
        )

#### 3. 데이터 가져오기

In [283]:
# 데이터 한 개 가져오기
response = table.get_item(
    Key={
        "user":"데스커",
        "category_id":50003683
    }
)
print(json.loads(response['Item']['brands']))

[['146395', '시디즈'], ['54', '한샘'], ['27883', '제닉스'], ['1179', '듀오백'], ['186140', '다니카'], ['235112', '에이픽스'], ['196418', '한성컴퓨터'], ['146393', '파트라'], ['229839', '비애노'], ['235875', '유어체어스'], ['25058', '리바트이즈마인'], ['580', '에넥스'], ['13130', '에넥스에니'], ['150152', '3RSYS'], ['137382', '블루밍홈'], ['205548', '린백'], ['205442', '베스툴'], ['176160', '체스툴'], ['175741', '커세어'], ['138590', '채우리'], ['11087', '데코라인']]


In [284]:
# 데이터 쿼리로 가져오기
response = table.query(KeyConditionExpression=Key('user').eq('데스커'))
items = response['Items']
print(items[:3])

[{'category_title': '책상', 'brands': '[["444", "\\uc774\\ucf00\\uc544"], ["205684", "\\ub9c8\\ucf13\\ube44"], ["54", "\\ud55c\\uc0d8"], ["137382", "\\ube14\\ub8e8\\ubc0d\\ud648"], ["205696", "\\uc18c\\ud504\\uc2dc\\uc2a4"], ["138213", "e\\uc2a4\\ub9c8\\ud2b8"], ["25058", "\\ub9ac\\ubc14\\ud2b8\\uc774\\uc988\\ub9c8\\uc778"], ["207029", "\\uc544\\uce74\\uc2dc\\uc544\\ub9ac\\ube59"], ["218624", "\\ub370\\uc2a4\\ucee4"], ["38271", "\\ud504\\ub9ac\\uba54\\uc774\\ub4dc"], ["117922", "\\ube44\\uc559\\uc2a4"], ["200864", "\\ubbf8\\uc98c\\ud558\\uc784"], ["17222", "\\ud544\\uc6f0"], ["11087", "\\ub370\\ucf54\\ub77c\\uc778"], ["29922", "\\ub450\\ub2f7"], ["13761", "\\uc77c\\ub8f8"], ["144168", "\\uc544\\uce74\\uc2dc\\uc544"], ["214444", "\\ub354\\uc900"], ["137388", "\\ud30c\\ub780\\ub4e4"], ["174958", "\\uc5d0\\uadf8\\uc2a4\\ud0c0"], ["20635", "FM\\ub514\\uc790\\uc778"]]', 'user': '데스커', 'category_id': Decimal('50001238')}, {'category_title': '의자', 'brands': '[["444", "\\uc774\\ucf00\\uc544"], [

In [234]:
# 데이터 스캔으로 가져오기
response = table.scan()
response['Items'];